In [48]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


#Import dataset
df = pd.read_csv('dataset/dataframe_after_exploratory_analysis_and_cleaning_data_1.csv', sep = ';', index_col = 0)


In [204]:
test_2016 = df.loc[df['year'] == 2016]
test_2017 = df.loc[df['year'] == 2017]

test = pd.concat([test_2016, test_2017])

train = df[~df.year.isin([2016,2017])]

X_train = train.iloc[:,1:5].values
y_train = train.iloc[:,0].values

X_test = test.iloc[:,1:5].values
y_test = test.iloc[:,0].values

label_encoder_city_code = LabelEncoder()
label_encoder_product = LabelEncoder()
label_encoder_product_type = LabelEncoder()

X_train[:,0] = label_encoder_city_code.fit_transform(X_train[:,0])
X_train[:,1] = label_encoder_product.fit_transform(X_train[:,1])
X_train[:,2] = label_encoder_product_type.fit_transform(X_train[:,2])

X_test[:,0] = label_encoder_city_code.fit_transform(X_test[:,0])
X_test[:,1] = label_encoder_product.fit_transform(X_test[:,1])
X_test[:,2] = label_encoder_product_type.fit_transform(X_test[:,2])

regressor_tree = DecisionTreeRegressor()
regressor_tree.fit(X_train, y_train)
regressor_tree.score(X_test, y_test)

#Predict the values

predictions = regressor_tree.predict(X_test)
test['predicted_area'] = predictions.tolist()
test.loc[:,'destinated_area'] = test.loc[:,'destinated_area'].map(lambda x: 0 if x < 0 else x)

In [38]:
import pandas as pd
import random

#Set seed
random.seed(10)

#Import dataset
dataset = pd.read_csv('dataset/dataframe_after_exploratory_analysis_and_cleaning_data_1.csv',index_col = 0, sep = ';')

#Cross Validation

#Separe the test dataset
auxiliar_test = dataset.loc[dataset.year.isin([2016,2017])]
dataset_without_2016_2017 = dataset.loc[~dataset.year.isin([2016,2017])]
temporary = auxiliar_test.loc[auxiliar_test.product_type == 'temporary']
permanent = auxiliar_test.loc[auxiliar_test.product_type == 'permanent']
pasture = auxiliar_test.loc[auxiliar_test.product_type == 'pasture']

#Temporary test
temporary_without_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
temporary_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
for year,sub_df in temporary.groupby('year'):
    position = random.sample([i for i in range(len(sub_df))],int(len(sub_df)*(0.05)))
    temporary_without_test_cases = pd.concat([temporary_without_test_cases,sub_df.iloc[list(set(range(len(sub_df)))-set(position)),:]])
    temporary_test_cases = pd.concat([temporary_test_cases,sub_df.iloc[position,:]])

#Permanent test
permanent_without_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
permanent_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
for year,sub_df in permanent.groupby('year'):
    position = random.sample([i for i in range(len(sub_df))],int(len(sub_df)*(0.05)))
    permanent_without_test_cases = pd.concat([permanent_without_test_cases,sub_df.iloc[list(set(range(len(sub_df)))-set(position)),:]])
    permanent_test_cases = pd.concat([permanent_test_cases,sub_df.iloc[position,:]])

#Pasture test
pasture_without_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
pasture_test_cases = pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year'])
for year,sub_df in pasture.groupby('year'):
    position = random.sample([i for i in range(len(sub_df))],int(len(sub_df)*(0.05)))
    pasture_without_test_cases = pd.concat([pasture_without_test_cases,sub_df.iloc[list(set(range(len(sub_df)))-set(position)),:]])
    pasture_test_cases = pd.concat([pasture_test_cases,sub_df.iloc[position,:]])

test = pd.concat([temporary_test_cases,permanent_test_cases,pasture_test_cases])
dataset = pd.concat([temporary_without_test_cases,permanent_without_test_cases,pasture_without_test_cases,dataset_without_2016_2017])

#Split dataset into 10 sub_dataset
temporary = dataset.loc[dataset.product_type == 'temporary']
permanent = dataset.loc[dataset.product_type == 'permanent']
pasture = dataset.loc[dataset.product_type == 'pasture']
size = int(len(dataset)*0.1)
df = []
for _ in range(10):
    df.append(pd.DataFrame([], columns = ['destinated_area', 'city_code', 'product', 'product_type','year']))
for product_type,sub_df_1 in dataset.groupby('product_type'):
    for year,sub_df_2 in sub_df_1.groupby('year'):
        positions_splited = []
        all_positions = [i for i in range(len(sub_df_2))]
        chosen_positions = set()
        for _ in range(9):
            aux = random.sample(list(set(all_positions)-chosen_positions),int(len(sub_df_2)*0.10))
            positions_splited.append(aux)
            chosen_positions = chosen_positions.union(aux)
        positions_splited.append(list(set(all_positions)-chosen_positions))
        for i in range(10):
            df[i] = pd.concat([df[i],sub_df_2.iloc[positions_splited[i],:]])
            dataset = pd.concat([dataset,sub_df_2.iloc[list(set(range(len(sub_df_2)))-set(positions_splited[i])),:]])
pd.concat(df[0:1] + df[2:])

,destinated_area,city_code,product,product_type,year
169,12325.07367,c129d0cd5f1afd51,Livestock,pasture,1997
276,41054.10663,e635f6b0f8a98808,Livestock,pasture,1997
199,302600.6298,cdc89121a4b9478b,Livestock,pasture,1997
853,223413.5841,7e56bd83460aabc7,Livestock,pasture,1997
906,13795.47913,85adcb13676b6172,Livestock,pasture,1997
...,...,...,...,...,...
34953,201.178344,68f01d67ad163f32,Beans,temporary,2017
34999,130.0,70a4c62beba3c168,Others,temporary,2017
35002,18.0,71975b563db10671,Beans,temporary,2017
35018,350.0,746cc42bfb8f6b62,Corn,temporary,2017


In [47]:
def wmape(df):
    real = df.iloc[:,0].values
    predict = df.iloc[:,5].values
    return abs(real-predict).sum()/real.sum()

cross_results = []
for i, value in enumerate(df):
    train = pd.concat(df[0:i] + df[i+1:])
    test_cross = df[i]
    print(test_cross)
    X_train = train.iloc[:,1:5].values
    y_train = train.iloc[:,0].values
    X_test = test_cross.iloc[:,1:5].values
    y_test = test_cross.iloc[:,0].values
    print(X_test)
    label_encoder_city_code = LabelEncoder()
    label_encoder_product = LabelEncoder()
    label_encoder_product_type = LabelEncoder()
    print(X_train)
    X_train[:,0] = label_encoder_city_code.fit_transform(X_train[:,0])
    X_train[:,1] = label_encoder_product.fit_transform(X_train[:,1])
    X_train[:,2] = label_encoder_product_type.fit_transform(X_train[:,2])
    print(X_test)
    X_test[:,0] = label_encoder_city_code.fit_transform(X_test[:,0])
    X_test[:,1] = label_encoder_product.fit_transform(X_test[:,1])
    X_test[:,2] = label_encoder_product_type.fit_transform(X_test[:,2])

    regressor_tree = DecisionTreeRegressor()
    regressor_tree.fit(X_train, y_train)
  
    #Predict the values
    predictions = regressor_tree.predict(X_test)
    test_cross['predicted_area'] = predictions.tolist()
    test_cross.loc[:,'destinated_area'] = test_cross.loc[:,'destinated_area'].map(lambda x: 0 if x < 0 else x)

    k = (wmape(test_cross.loc[test_cross['product_type'] == 0]) + \
    wmape(test_cross.loc[test_cross['product_type'] == 1]) + \
    wmape(test_cross.loc[test_cross['product_type'] == 2]))/3
    cross_results.append(k)


 




      destinated_area city_code    product product_type  year
169       12325.07367       119  Livestock      pasture  1997
276       41054.10663       132  Livestock      pasture  1997
199       302600.6298       122  Livestock      pasture  1997
853       223413.5841        74  Livestock      pasture  1997
906       13795.47913        81  Livestock      pasture  1997
...               ...       ...        ...          ...   ...
34315            43.0       107     Others    temporary  2017
34696       57.223284        25       Rice    temporary  2017
35285       58.909091        96      Beans    temporary  2017
34793            22.0        38      Beans    temporary  2017
34784            50.0        36       Rice    temporary  2017

[3552 rows x 5 columns]
[[119 'Livestock' 'pasture' 1997]
 [132 'Livestock' 'pasture' 1997]
 [122 'Livestock' 'pasture' 1997]
 ...
 [96 'Beans' 'temporary' 2017]
 [38 'Beans' 'temporary' 2017]
 [36 'Rice' 'temporary' 2017]]
[['f3c2cc112eb0e6fe' 'Livestock

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']

In [40]:
def wmape(df):
    real = df.iloc[:,0].values
    predict = df.iloc[:,5].values
    return abs(real-predict).sum()/real.sum()

In [235]:
#Calculate the error
(wmape(test.loc[test['product_type'] == 'pasture']) + \
wmape(test.loc[test['product_type'] == 'temporary']) + \
wmape(test.loc[test['product_type'] == 'permanent']))/3

IndexError: single positional indexer is out-of-bounds

In [200]:
wmape(test.loc[test['product_type'] == 'pasture'])

0.03795314030092393

In [202]:
wmape(test.loc[test['product_type'] == 'temporary'])

0.2458887881271535

In [203]:
wmape(test.loc[test['product_type'] == 'permanent'])

0.14532473435099405